# Sistema de Recomendación

In [32]:
import pandas as pd
import numpy as np
import string 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
import nltk as nlt
from fastapi import HTTPException

In [33]:
df = pd.read_parquet(r'C:\Users\GASTON\Desktop\PI1\DATA\df_mlops.parquet')
#Creamos un DataFrame a partir del dataset previamente depurado

In [34]:
df.head(3)

,id,spoken_languagesname,genresname,title,overview,vote_average,popularity,status,production_countriesname,release_year,director,actor
2,10001,English,"Science Fiction, Comedy",Young Einstein,Albert Einstein is the son of a Tasmanian appl...,4.5,2.562888,Released,Australia,1988,Yahoo Serious,"Yahoo Serious, Odile Le Clezio, Peewee Wilson,..."
3,100010,English,"Drama, War",Flight Command,"A rookie flyer, Ens. Alan Drake, joins the fam...",6.0,0.769266,Released,United States of America,1940,Frank Borzage,"Robert Taylor, Ruth Hussey, Walter Pidgeon, Pa..."
4,100017,Deutsch,Drama,Hounded,Deals with the obsessive relationship between ...,4.8,2.964103,Released,Germany,2006,Angelina Maccarone,"Kostja Ullmann, Maren Kroymann, Moritz Grove, ..."


In [35]:
#Como primer paso, debemos limpiar y tokenizar el texto.
#El primer elemento a utilizar para obtener una cadena de palabras clave, serán las stopwords (identificando las palabras adicionales).
# Se definirán las stopwords para los idiomas de inglés y español, convirtiendolas en sets

nlt.download('stopwords')

en_stopwords = set(stopwords.words('english'))
esp_stopwords = set(stopwords.words('spanish'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\GASTON\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [36]:
# Función para limpiar y tokenizar nuestras columnas
def limpiarTexto(cadena):
    '''
    Esta función tokeniza el texto, elimina signos de puntuación y convierte los caracteres en minúsculas.
    Luego, filtra las stopwords.
    Espera como parámetro una cadena de texto.
    '''
    # Tokenización
    etiquetas = word_tokenize(cadena) #se convierte la cadena en una lista de palabras
    etiquetas = [word.lower() for word in etiquetas if word.isalpha()] # Se eliminan  signos de puntuación y convierte todo a minúsculas con lower()
    etiquetas = [word for word in etiquetas if word not in en_stopwords and word not in esp_stopwords] # Se quitan stopwords en inglés y español descargadas en el paso anterior
    return etiquetas

In [10]:
#import nltk
#nltk.data.path.append('C:/Users/GASTON/AppData/Roaming/nltk_data')

In [11]:
#import nltk
#nltk.data.path = ['C:/Users/GASTON/AppData/Roaming/nltk_data']
#nltk.download('punkt', download_dir='C:/Users/GASTON/AppData/Roaming/nltk_data')
#Fue necesario hacer estas modificaciones para que funcione la galería nltk


[nltk_data] Downloading package punkt to
[nltk_data]     C:/Users/GASTON/AppData/Roaming/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [37]:
df['title_depurado'] = df['title'].apply(lambda x: limpiarTexto(x))
df['overview_depurado'] = df['overview'].dropna().apply(lambda x: limpiarTexto(x))
df.head(3)

,id,spoken_languagesname,genresname,title,overview,vote_average,popularity,status,production_countriesname,release_year,director,actor,title_depurado,overview_depurado
2,10001,English,"Science Fiction, Comedy",Young Einstein,Albert Einstein is the son of a Tasmanian appl...,4.5,2.562888,Released,Australia,1988,Yahoo Serious,"Yahoo Serious, Odile Le Clezio, Peewee Wilson,...","[young, einstein]","[albert, einstein, tasmanian, apple, farmer, d..."
3,100010,English,"Drama, War",Flight Command,"A rookie flyer, Ens. Alan Drake, joins the fam...",6.0,0.769266,Released,United States of America,1940,Frank Borzage,"Robert Taylor, Ruth Hussey, Walter Pidgeon, Pa...","[flight, command]","[rookie, flyer, ens, alan, drake, joins, famou..."
4,100017,Deutsch,Drama,Hounded,Deals with the obsessive relationship between ...,4.8,2.964103,Released,Germany,2006,Angelina Maccarone,"Kostja Ullmann, Maren Kroymann, Moritz Grove, ...",[hounded],"[deals, obsessive, relationship, confused, tee..."


In [38]:
df = df.drop(['overview'],axis=1) #La columna overview (Pequeño resumen de la película) ha sido depurada y agregadas
#sus palabras clave a overview_depurado, por lo que se puede eliminar. No así el título, que será el retorno del 
#sistema de recomendación.

In [39]:
# La columna overview_depurado presenta algunos campos que no son strings. 
# Por eso, convertimos esa columna, la de título, título depurado y la de género en texto.
df['overview_depurado'] = df['overview_depurado'].astype(str)
df['title'] = df['title'].astype(str)
df['title_depurado'] = df['title_depurado'].astype(str)
df['genresname'] = df['genresname'].astype(str)

Ahora que se cuenta con los campos específicos, es preciso generar una columna que contenga la información de todos ellos. Esto simplificará la vectorización y análisis (en comparación a trabajar con varias columnas).
Se busca lograr una representación completa de la película, agregando las palabras clave del título, género y descripción. Además, se ponderará o dará mayor valor al título y al género, para que tengan más importancia, destacándose en la matriz TF-IDF para mejorar la precisión de las recomendaciones.

In [40]:
df['descripcion_combinada'] = df['title_depurado']+ ' '+ df['title_depurado'] + ' ' + df['overview_depurado'] + ' ' + df['genresname']+ ' ' + df['genresname']+ ' ' + df['genresname']
#Se crea la columna "descripción combinada", que contiene los valores del título (depurado), el resumen (depurado) y el género.
# Como se dijo, se repiten los valores del título y género para que tengan mayor fuerza.

In [41]:
# Para realizar la vectorización, creamos una lista de stopwords de los dos idiomas
stopwords_todas = list(set(en_stopwords).union(set(esp_stopwords)))

Ahora que tenemos todos los valores, debemos vectorizar y calcular la similitud del coseno para definir nuestra función de recomendación.
La vectorización se realiza con TFIDFVectorizer, con todas las stopwords guardadas.

In [42]:
# Vectorización utilizando TF-IDF con stop words en inglés y español
vectorizacion = TfidfVectorizer(stop_words=stopwords_todas, max_features=5000) #Se definen los paráemetros para convertir el texto en una matriz.
matrizTFIDF = vectorizacion.fit_transform(df['descripcion_combinada']) #Se crea una matriz TF-IDF sobre la columna "descripción combinada"
print(matrizTFIDF.shape) #dimensiones de la matriz

(34318, 5000)


In [43]:
# Realizamos la reducción de dimensionalidad
val_singulares = TruncatedSVD(n_components=100) #Se reduce la dimensionalidad de la matriz a 100 dimensiones, basándose en los valores singulares más altos.
matriz_sing = val_singulares.fit_transform(matrizTFIDF)#Aplica el ajuste y la transformación a la matriz TF-IDF
print(matriz_sing.shape) #Nuevas dimensiones sobre los valores singulares más importantes

(34318, 100)


La reducción de la dimensionalidad genera una notable mejora de eficacia. Al eliminarse los componentes principales, se eliminan los menos significativos, que podrían introducir ruido al análisis.
Además, la menor dimensionalidad contribuye con la mejora del rendimiento y la capacidad de generalización del modelo.

In [44]:
# Ahora debemos calcular la similitud del coseno entre las filas de la matriz reducida (esto permitirá encontrar
#  películas similares en función de sus características combinadas: título, descripción y género). 
# Además, permite analizar relaciones entre películas, lo que ayuda a mejorar la precisión del sistema de recomendación.

similitud_coseno = cosine_similarity(matriz_sing, matriz_sing) #Para buscar la similitud, cruzamos dos veces la matriz.

Ahora estamos en condiciones de crear la función de recomendación.

In [45]:
def recomendacion(title, df=df, similitud_coseno=similitud_coseno):
    try:
        indice = df[df['title'].str.lower() == title.lower()].index[0] # Se define el índice de película similar
        similitud = list(enumerate(similitud_coseno[indice])) 
        similitud = sorted(similitud, key=lambda x: x[1], reverse=True) #Se obtiene el score de similitud y se ordenan.
        # Se obtienen las películas similares, dejando afuera la primera(que es el mismo título):
        similitud = similitud[1:6]
        indices_similares = [i[0] for i in similitud]
        peliculas_similares = df['title'].iloc[indices_similares].tolist() #Se crea una lista con las películas
        mensaje = f"Se encontraron los siguientes títulos similares a '{title}':"
        print(mensaje)
        # Retornar la lista de títulos similares
        return peliculas_similares
    except IndexError:
        print(f"No se encontró la película '{title}'. Por favor, intente con otro título.")

In [47]:
test = recomendacion('toy story 3')
test

Se encontraron los siguientes títulos similares a 'toy story 3':


['Yesterday Girl', 'The Muse', 'Scream of the Ants', 'Nuts', 'Ritual']

In [48]:
test2 = recomendacion('Young Einstein')
test2

Se encontraron los siguientes títulos similares a 'Young Einstein':


['About Cherry',
 'Virginia',
 'Wintersleepers',
 'Riot on Sunset Strip',
 'The Saviour']

In [49]:
test3= recomendacion('Star Wars')
test3

Se encontraron los siguientes títulos similares a 'Star Wars':


['The Spiderwick Chronicles',
 'Escape to Witch Mountain',
 'Paws',
 'The Prince and the Pauper',
 'Harry Potter and the Prisoner of Azkaban']

In [50]:
test_error = recomendacion('askedddego')
test_error

No se encontró la película 'askedddego'. Por favor, intente con otro título.


En este punto, encontramos problemas de memoria para deployar el Sistema de Recomendación. Por ese motivo, se van a ajustar y reducir el dataset.

In [51]:
# En primer lugar, vamos a ordenar el DataFrame por 'vote_average' en orden descendente
df_sorted = df.sort_values(by='popularity', ascending=False)
df_sorted.head()

,id,spoken_languagesname,genresname,title,vote_average,popularity,status,production_countriesname,release_year,director,actor,title_depurado,overview_depurado,descripcion_combinada
13856,211672,English,"Comedy, Animation, Adventure, Family",Minions,6.4,547.488298,Released,United States of America,2015,"Kyle Balda, Pierre Coffin","Sandra Bullock, Jon Hamm, Michael Keaton, Alli...",['minions'],"['minions', 'stuart', 'kevin', 'bob', 'recruit...","['minions'] ['minions'] ['minions', 'stuart', ..."
21460,297762,"English, Deutsch","Fantasy, Action, Adventure",Wonder Woman,7.2,294.337037,Released,United States of America,2017,Patty Jenkins,"Gal Gadot, Chris Pine, Robin Wright, Danny Hus...","['wonder', 'woman']","['amazon', 'princess', 'comes', 'world', 'man'...","['wonder', 'woman'] ['wonder', 'woman'] ['amaz..."
23352,321612,English,"Fantasy, Romance, Family",Beauty and the Beast,6.8,287.253654,Released,United Kingdom,2017,Bill Condon,"Emma Watson, Dan Stevens, Luke Evans, Kevin Kl...","['beauty', 'beast']","['adaptation', 'disney', 'version', 'classic',...","['beauty', 'beast'] ['beauty', 'beast'] ['adap..."
24696,339403,English,"Action, Crime",Baby Driver,7.2,228.032744,Released,United Kingdom,2017,Edgar Wright,"Ansel Elgort, Lily James, Kevin Spacey, Jamie ...","['baby', 'driver']","['coerced', 'working', 'crime', 'boss', 'young...","['baby', 'driver'] ['baby', 'driver'] ['coerce..."
10545,177572,English,"Animation, Adventure, Family, Action, Comedy",Big Hero 6,7.8,213.849907,Released,United States of America,2014,"Chris Williams, Don Hall","Scott Adsit, Ryan Potter, Daniel Henney, T.J. ...","['big', 'hero']","['special', 'bond', 'develops', 'inflatable', ...","['big', 'hero'] ['big', 'hero'] ['special', 'b..."


In [52]:
# Ahora realizaremos un ranking de las 6 mil películas con mejor ranking
df_top = df_sorted.head(6000)
df_top.head()

,id,spoken_languagesname,genresname,title,vote_average,popularity,status,production_countriesname,release_year,director,actor,title_depurado,overview_depurado,descripcion_combinada
13856,211672,English,"Comedy, Animation, Adventure, Family",Minions,6.4,547.488298,Released,United States of America,2015,"Kyle Balda, Pierre Coffin","Sandra Bullock, Jon Hamm, Michael Keaton, Alli...",['minions'],"['minions', 'stuart', 'kevin', 'bob', 'recruit...","['minions'] ['minions'] ['minions', 'stuart', ..."
21460,297762,"English, Deutsch","Fantasy, Action, Adventure",Wonder Woman,7.2,294.337037,Released,United States of America,2017,Patty Jenkins,"Gal Gadot, Chris Pine, Robin Wright, Danny Hus...","['wonder', 'woman']","['amazon', 'princess', 'comes', 'world', 'man'...","['wonder', 'woman'] ['wonder', 'woman'] ['amaz..."
23352,321612,English,"Fantasy, Romance, Family",Beauty and the Beast,6.8,287.253654,Released,United Kingdom,2017,Bill Condon,"Emma Watson, Dan Stevens, Luke Evans, Kevin Kl...","['beauty', 'beast']","['adaptation', 'disney', 'version', 'classic',...","['beauty', 'beast'] ['beauty', 'beast'] ['adap..."
24696,339403,English,"Action, Crime",Baby Driver,7.2,228.032744,Released,United Kingdom,2017,Edgar Wright,"Ansel Elgort, Lily James, Kevin Spacey, Jamie ...","['baby', 'driver']","['coerced', 'working', 'crime', 'boss', 'young...","['baby', 'driver'] ['baby', 'driver'] ['coerce..."
10545,177572,English,"Animation, Adventure, Family, Action, Comedy",Big Hero 6,7.8,213.849907,Released,United States of America,2014,"Chris Williams, Don Hall","Scott Adsit, Ryan Potter, Daniel Henney, T.J. ...","['big', 'hero']","['special', 'bond', 'develops', 'inflatable', ...","['big', 'hero'] ['big', 'hero'] ['special', 'b..."


In [53]:
df_top.columns

Index(['id', 'spoken_languagesname', 'genresname', 'title', 'vote_average',
       'popularity', 'status', 'production_countriesname', 'release_year',
       'director', 'actor', 'title_depurado', 'overview_depurado',
       'descripcion_combinada'],
      dtype='object')

In [57]:
columnas_out = ['spoken_languagesname', 'vote_average', 'popularity', 'status', 'production_countriesname', 'release_year', 'director', 'actor']

In [60]:
df_top=df_top.drop(columnas_out, axis=1)
df_top.head(1)

,id,genresname,title,title_depurado,overview_depurado,descripcion_combinada
13856,211672,"Comedy, Animation, Adventure, Family",Minions,['minions'],"['minions', 'stuart', 'kevin', 'bob', 'recruit...","['minions'] ['minions'] ['minions', 'stuart', ..."


In [68]:
# Vectorización utilizando TF-IDF con stop words en inglés y español
vectorizacion = TfidfVectorizer(stop_words=stopwords_todas, max_features=5000) #Se definen los paráemetros para convertir el texto en una matriz.
matrizTFIDF = vectorizacion.fit_transform(df_top['descripcion_combinada']) #Se crea una matriz TF-IDF sobre la columna "descripción combinada"
print(matrizTFIDF.shape) #dimensiones de la matriz

(6000, 5000)


In [69]:
# Realizamos la reducción de dimensionalidad
val_singulares = TruncatedSVD(n_components=100) #Se reduce la dimensionalidad de la matriz a 100 dimensiones, basándose en los valores singulares más altos.
matriz_sing = val_singulares.fit_transform(matrizTFIDF)#Aplica el ajuste y la transformación a la matriz TF-IDF
print(matriz_sing.shape) #Nuevas dimensiones sobre los valores singulares más importantes

(6000, 100)


In [70]:
# Ahora debemos calcular la similitud del coseno entre las filas de la matriz reducida (esto permitirá encontrar
#  películas similares en función de sus características combinadas: título, descripción y género). 
# Además, permite analizar relaciones entre películas, lo que ayuda a mejorar la precisión del sistema de recomendación.

similitud_coseno = cosine_similarity(matriz_sing, matriz_sing) #Para buscar la similitud, cruzamos dos veces la matriz.

In [79]:
def recomendacion(title, df=df_top, similitud_coseno=similitud_coseno):
    try:
        indice = df_top[df_top['title'].str.lower() == title.lower()].index[0] # Se define el índice de película similar
        similitud = list(enumerate(similitud_coseno[indice])) 
        similitud = sorted(similitud, key=lambda x: x[1], reverse=True) #Se obtiene el score de similitud y se ordenan.
        # Se obtienen las películas similares, dejando afuera la primera(que es el mismo título):
        similitud = similitud[1:6]
        indices_similares = [i[0] for i in similitud]
        peliculas_similares = df['title'].iloc[indices_similares].tolist() #Se crea una lista con las películas
        mensaje = f"Se encontraron los siguientes títulos similares a '{title}':"
        print(mensaje)
        # Retornar la lista de títulos similares
        return peliculas_similares
    except IndexError:
        print(f"No se encontró la película '{title}'. Por favor, intente con otro título.")

In [ ]:
indice = df_top[df_top['title'].str.strip().str.lower() == title.strip().lower()].index[0]

In [73]:
test4= recomendacion('Star wars')
test4

Se encontraron los siguientes títulos similares a 'Star wars':


['Raiders of the Lost Ark',
 'Operation Mekong',
 'Call of Heroes',
 'Indiana Jones and the Temple of Doom',
 'The Quest']

In [89]:
test5=recomendacion('toy story 3')
test5

Se encontraron los siguientes títulos similares a 'toy story 3':


['Drinking Buddies', 'About Alex', 'Reality Bites', 'Birdman', 'Rough Night']

In [31]:
test_error2=recomendacion('asdasd')
test_error2

No se encontró la película 'asdasd'. Por favor, intente con otro título.


In [91]:
df_top.to_parquet(r'C:\Users\GASTON\Desktop\PI1\DATA\recomendacion.parquet',index=True)

In [92]:
prueba = pd.read_parquet(r'C:\Users\GASTON\Desktop\PI1\DATA\recomendacion.parquet')
prueba.head()

,id,genresname,title,title_depurado,overview_depurado,descripcion_combinada
13856,211672,"Comedy, Animation, Adventure, Family",Minions,['minions'],"['minions', 'stuart', 'kevin', 'bob', 'recruit...","['minions'] ['minions'] ['minions', 'stuart', ..."
21460,297762,"Fantasy, Action, Adventure",Wonder Woman,"['wonder', 'woman']","['amazon', 'princess', 'comes', 'world', 'man'...","['wonder', 'woman'] ['wonder', 'woman'] ['amaz..."
23352,321612,"Fantasy, Romance, Family",Beauty and the Beast,"['beauty', 'beast']","['adaptation', 'disney', 'version', 'classic',...","['beauty', 'beast'] ['beauty', 'beast'] ['adap..."
24696,339403,"Action, Crime",Baby Driver,"['baby', 'driver']","['coerced', 'working', 'crime', 'boss', 'young...","['baby', 'driver'] ['baby', 'driver'] ['coerce..."
10545,177572,"Animation, Adventure, Family, Action, Comedy",Big Hero 6,"['big', 'hero']","['special', 'bond', 'develops', 'inflatable', ...","['big', 'hero'] ['big', 'hero'] ['special', 'b..."
